# **Homework 5**

In your project, you will pick an image dataset to solve a classification task. Provide a link to your dataset.

Pokemon Image Dataset: https://www.kaggle.com/vishalsubbiah/pokemon-images-and-types?select=images

## **Task 1**

### **Pre-Setup**

I'd like to find out each pokemons primary type(fire, water, grass etc.) based on their image.

In [1]:
# Imports
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
import tensorflow as tf

In [2]:
from tensorflow import keras as ks
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, Activation, BatchNormalization

In [4]:
# Get CSV Data
df = pd.read_csv("pokemon.csv")
df.head(5)

,Name,Type1,Type2
0,bulbasaur,Grass,Poison
1,ivysaur,Grass,Poison
2,venusaur,Grass,Poison
3,charmander,Fire,NaN
4,charmeleon,Fire,NaN


In [5]:
# Select only a few types of pokemon because data are too little
selected = ["Water", "Fire", "Grass"]

df = df[df['Type1'].isin(selected)]
df['Type1'].value_counts()

Water    114
Grass     78
Fire      53
Name: Type1, dtype: int64

In [6]:
img = os.listdir('./images')
for i in img:
  name = i.split('.')[0]
  df.loc[df['Name'] == name, ["Image"]] = i
df.head(5)

,Name,Type1,Type2,Image
0,bulbasaur,Grass,Poison,bulbasaur.png
1,ivysaur,Grass,Poison,ivysaur.png
2,venusaur,Grass,Poison,venusaur.png
3,charmander,Fire,NaN,charmander.png
4,charmeleon,Fire,NaN,charmeleon.png


In [7]:
import shutil
shutil.rmtree('train/')
shutil.rmtree('test/')
shutil.rmtree('valid/')

os.mkdir('train/')
os.mkdir('test/')
os.mkdir('valid/')

for i in df['Type1'].unique():
  os.mkdir('train/'+str(i)+'/')
  os.mkdir('test/'+str(i)+'/')
  os.mkdir('valid/'+str(i)+'/')

In [8]:
x_train, x_rem, y_train, y_rem = train_test_split(df, df["Type1"], train_size=0.7)
x_valid, x_test, y_valid, y_test = train_test_split(x_rem, y_rem, test_size=0.5)

In [10]:
from shutil import copyfile, copy2
for image, types in zip("./images/" + x_train['Image'], y_train):
  copy2(image, 'train/' + types)
for image, types in zip("./images/" + x_test['Image'], y_test):
  copy2(image, 'test/' + types)
for image, types in zip("./images/" + x_valid['Image'], y_valid):
  copy2(image, 'valid/' + types)

In [11]:
train = ImageDataGenerator().flow_from_directory('train/')
test = ImageDataGenerator().flow_from_directory('test/')
val = ImageDataGenerator().flow_from_directory('valid/')

Found 171 images belonging to 3 classes.
Found 37 images belonging to 3 classes.
Found 37 images belonging to 3 classes.


**Part 1 (20 points):** This step involves downloading, preparing, and visualizing your dataset. Create a convolutional base using a common pattern: a stack of Conv and MaxPooling layers. Depending on the problem and the dataset you must decide what pattern you want to use (i.e., how many Conv layers and how many pooling layers). Please describe why you chose a particular pattern. Add the final dense layer(s). Compile and train the model. Report the final evaluation and describe the metrics.

---

For the first Conv2D I choosed 16 filters with size 3, the activation function for each Conv2D is relu. To reduce the spatial dimensions of the output volume a MaxPooling2D is used where the size is 3. For the third layer a dropout of 30% is used to prevent overfitting. After that add the 2nd, 3rd convolutional layer and increase the filter gradually. Lastly flatten into 1 dimension and a dense layer.

In [13]:
model = Sequential()

model.add(Conv2D(16, (3, 3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (3, 3)))

model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (3, 3)))

model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (3, 3)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(3, activation = 'softmax'))
model.compile(loss = 'mean_squared_error', optimizer = "adam", metrics = ['accuracy'])

In [14]:
history = model.fit(train, epochs = 20, validation_data = val)

/usr/local/lib/python3.9/site-packages/PIL/Image.py:973: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
2021-11-06 02:10:09.838750: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20
6/6 [==============================] - 3s 370ms/step - loss: 0.3063 - accuracy: 0.3860 - val_loss: 0.4370 - val_accuracy: 0.2973
Epoch 2/20
6/6 [==============================] - 2s 335ms/step - loss: 0.2407 - accuracy: 0.5029 - val_loss: 0.3736 - val_accuracy: 0.3784
Epoch 3/20
6/6 [==============================] - 2s 353ms/step - loss: 0.2036 - accuracy: 0.5965 - val_loss: 0.3821 - val_accuracy: 0.4054
Epoch 4/20
6/6 [==============================] - 2s 322ms/step - loss: 0.1829 - accuracy: 0.6608 - val_loss: 0.3371 - val_accuracy: 0.4865
Epoch 5/20
6/6 [==============================] - 2s 332ms/step - loss: 0.1405 - accuracy: 0.7368 - val_loss: 0.3085 - val_accuracy: 0.5135
Epoch 6/20
6/6 [==============================] - 2s 325ms/step - loss: 0.1239 - accuracy: 0.7661 - val_loss: 0.2952 - val_accuracy: 0.5405
Epoch 7/20
6/6 [==============================] - 2s 334ms/step - loss: 0.1096 - accuracy: 0.7836 - val_loss: 0.2609 - val_accuracy: 0.5405
Epoch 8/20
6/6 [====

In [15]:
from sklearn.metrics import classification_report
predict = model.predict(test)
y_pred = np.argmax(predict, axis=-1)

y_pred_conv = []
for i in y_pred:
  if i == 0:
    y_pred_conv.append("Fire")
  elif i == 1:
    y_pred_conv.append("Grass")
  elif i == 2:
    y_pred_conv.append("Water")

print(classification_report(y_test, y_pred_conv))

              precision    recall  f1-score   support

        Fire       0.25      0.44      0.32         9
       Grass       0.33      0.23      0.27        13
       Water       0.50      0.40      0.44        15

    accuracy                           0.35        37
   macro avg       0.36      0.36      0.35        37
weighted avg       0.38      0.35      0.35        37



The main matrices is precision, but recall and f1 score is also printed out for each pokemon type. We can see that water type pokemon have much higher precision than fire and grass type, one reason is because water type pokemon have the most number of data thus it have trained better than the other two.

**Part 2 (25 points):** The following models are widely used for transfer learning because of their performance and architectural innovations:

1. VGG (e.g., VGG16 or VGG19).
2. GoogLeNet (e.g., InceptionV3).
3. Residual Network (e.g., ResNet50).
4. MobileNet (e.g., MobileNetV2)

Choose any **one** of the above models to perform the classification task you did in Part 1. Evaluate the results using the same metrics as in Part 1. Are there any differences? Why or why not?

---

In [16]:
from keras.applications.vgg16 import VGG16
from keras.models import Model

model = VGG16(include_top=False, input_shape=(256, 256, 3))
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(64, activation='relu')(flat1)
output = Dense(3, activation='softmax')(class1)
model = Model(inputs=model.inputs, outputs=output)
model.summary()

58900480/58889256 [==============================] - 2s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
______________________________________________________________

In [17]:
model.compile(loss = 'mean_squared_error', optimizer = "adam", metrics = ['accuracy'])
history = model.fit(train, epochs = 5, validation_data = val)

/usr/local/lib/python3.9/site-packages/PIL/Image.py:973: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/5
6/6 [==============================] - 84s 14s/step - loss: 0.3829 - accuracy: 0.4211 - val_loss: 0.3243 - val_accuracy: 0.5135
Epoch 2/5
6/6 [==============================] - 80s 15s/step - loss: 0.3548 - accuracy: 0.4678 - val_loss: 0.3243 - val_accuracy: 0.5135
Epoch 3/5
6/6 [==============================] - 80s 13s/step - loss: 0.3548 - accuracy: 0.4678 - val_loss: 0.3243 - val_accuracy: 0.5135
Epoch 4/5
6/6 [==============================] - 81s 13s/step - loss: 0.3548 - accuracy: 0.4678 - val_loss: 0.3243 - val_accuracy: 0.5135
Epoch 5/5
6/6 [==============================] - 81s 13s/step - loss: 0.3548 - accuracy: 0.4678 - val_loss: 0.3243 - val_accuracy: 0.5135


In [18]:
predict = model.predict(test)
y_pred = np.argmax(predict, axis=-1)

y_pred_conv = []
for i in y_pred:
  if i == 0:
    y_pred_conv.append("Fire")
  elif i == 1:
    y_pred_conv.append("Grass")
  elif i == 2:
    y_pred_conv.append("Water")

print(classification_report(y_test, y_pred_conv))

              precision    recall  f1-score   support

        Fire       0.00      0.00      0.00         9
       Grass       0.00      0.00      0.00        13
       Water       0.41      1.00      0.58        15

    accuracy                           0.41        37
   macro avg       0.14      0.33      0.19        37
weighted avg       0.16      0.41      0.23        37



/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The result is worse than the previous one, identifying Water type pokemon is slightly lower and the non of the tests for both fire & grass type pokemon is correct. I'd say there are few reasons behind this:

1. the dataset is two small, there are only ~230 images of pokemon thus the accuracy may not go up.
2. Test set is too small as well, when splitting up the data set, only 37 images were put into test set.
3. Overfitting may be one of the cause.
4. Pokemons of each type do not have obvious features except for their colors(red, green, blue), and even some pokemons' color do not match their actual type.

**Part 3 (25 points):** Use data augmentation to increase the diversity of your dataset by applying random transformations such as image rotation (you can use any other technique as well). Repeat the process from part 1 with this augmented data. Did you observe any difference in results?

---

In [19]:
from tensorflow.keras import layers
resize_rescale_flip_rotation = tf.keras.Sequential([
  layers.Resizing(128, 128),
  layers.Rescaling(1./255),
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

In [21]:
model = tf.keras.Sequential([
  # Add the preprocessing layers you created earlier.
  resize_rescale_flip_rotation,
  layers.Conv2D(16, (3, 3), activation = 'relu'),
  layers.MaxPool2D(pool_size = (3, 3)),

  Conv2D(32, (3, 3), activation = 'relu'),
  MaxPool2D(pool_size = (3, 3)),
    
  Conv2D(64, (3, 3), activation = 'relu'),
  BatchNormalization(),
  MaxPool2D(pool_size = (3, 3)),
  Dropout(0.3),

  Flatten(),
  Dense(256, activation = 'relu'),
  BatchNormalization(),
  Dropout(0.5),
  Dense(3, activation = 'softmax')
])

model.compile(loss = 'mean_squared_error', optimizer = "adam", metrics = ['accuracy'])

In [90]:
history = model.fit(train, epochs = 20, validation_data = val)

Epoch 1/20
1/6 [====>.........................] - ETA: 0s - loss: 0.0574 - accuracy: 0.9091

/usr/local/lib/python3.9/site-packages/PIL/Image.py:973: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


6/6 [==============================] - 1s 153ms/step - loss: 0.1009 - accuracy: 0.8129 - val_loss: 0.2850 - val_accuracy: 0.4054
Epoch 2/20
6/6 [==============================] - 1s 128ms/step - loss: 0.0785 - accuracy: 0.8480 - val_loss: 0.1992 - val_accuracy: 0.5676
Epoch 3/20
6/6 [==============================] - 1s 122ms/step - loss: 0.0787 - accuracy: 0.8538 - val_loss: 0.1635 - val_accuracy: 0.6757
Epoch 4/20
6/6 [==============================] - 1s 118ms/step - loss: 0.0761 - accuracy: 0.8713 - val_loss: 0.1858 - val_accuracy: 0.5946
Epoch 5/20
6/6 [==============================] - 1s 123ms/step - loss: 0.0737 - accuracy: 0.8596 - val_loss: 0.1622 - val_accuracy: 0.6486
Epoch 6/20
6/6 [==============================] - 1s 120ms/step - loss: 0.0768 - accuracy: 0.8655 - val_loss: 0.1693 - val_accuracy: 0.5946
Epoch 7/20
6/6 [==============================] - 1s 135ms/step - loss: 0.0900 - accuracy: 0.8363 - val_loss: 0.1154 - val_accuracy: 0.7838
Epoch 8/20
6/6 [===============

In [97]:
predict = model.predict(test)
y_pred = np.argmax(predict, axis=-1)

y_pred_conv = []
for i in y_pred:
  if i == 0:
    y_pred_conv.append("Fire")
  elif i == 1:
    y_pred_conv.append("Grass")
  elif i == 2:
    y_pred_conv.append("Water")

print(classification_report(y_test, y_pred_conv))

              precision    recall  f1-score   support

        Fire       0.33      0.22      0.27         9
       Grass       0.50      0.85      0.63        13
       Water       0.78      0.47      0.58        15

    accuracy                           0.54        37
   macro avg       0.54      0.51      0.49        37
weighted avg       0.57      0.54      0.52        37



/usr/local/lib/python3.9/site-packages/PIL/Image.py:973: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Comparing to the result in task 1 the result I have here is much better, I've reduce the size of the image by half and also randomly flipped & rotate the images.
Most of the tests for water type pokemon is correct and tests for both fire&grass type pokemon have substantially higher precision.